In [1]:
import pandas as pd

In [2]:
traffic_crashes = pd.read_csv(r"C:\Users\lenovo\Downloads\Traffic_Crashes_-_Crashes.csv")

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
traffic_crashes_copy = traffic_crashes.copy()

In [5]:
traffic_crashes.head()

,CRASH_RECORD_ID,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,REPORT_TYPE,CRASH_TYPE,INTERSECTION_RELATED_I,NOT_RIGHT_OF_WAY_I,HIT_AND_RUN_I,DAMAGE,DATE_POLICE_NOTIFIED,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_NO,STREET_DIRECTION,STREET_NAME,BEAT_OF_OCCURRENCE,PHOTOS_TAKEN_I,STATEMENTS_TAKEN_I,DOORING_I,WORK_ZONE_I,WORK_ZONE_TYPE,WORKERS_PRESENT_I,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,000c4307d8e9b39075cffdd0aade3603e0f96f14e41da9...,NaN,01/14/2025 12:25:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,SNOW,DAYLIGHT,SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN (NOT RAISED),NaN,STRAIGHT AND LEVEL,SNOW OR SLUSH,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,NaN,NaN,Y,"$501 - $1,500",01/14/2025 12:38:00 PM,IMPROPER TURNING/NO SIGNAL,IMPROPER OVERTAKING/PASSING,6352,N,SHERIDAN RD,2433.0,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,12,3,1,41.997808,-87.655770,POINT (-87.655770494712 41.997807727633)
1,027b0b4c21460d3441fd83929abb9673c6fc0c7d575675...,NaN,05/23/2025 09:30:00 AM,30,STOP SIGN/FLASHER,UNKNOWN,UNKNOWN,DAYLIGHT,TURNING,DIVIDED - W/MEDIAN (NOT RAISED),NaN,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,NaN,NaN,Y,"OVER $1,500",05/27/2025 10:40:00 AM,IMPROPER OVERTAKING/PASSING,IMPROPER OVERTAKING/PASSING,3555,N,WESTERN AVE,1921.0,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,9,6,5,41.946529,-87.688106,POINT (-87.688106391039 41.946529480518)
2,04d91dffc94f677358ca47056921ba5c4224320df27ed4...,Y,04/05/2025 08:00:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,UNKNOWN,PARKED MOTOR VEHICLE,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,NaN,NaN,Y,"OVER $1,500",04/05/2025 09:23:00 PM,UNABLE TO DETERMINE,UNABLE TO DETERMINE,1005,N,DRAKE AVE,1121.0,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20,7,4,41.899325,-87.715074,POINT (-87.715074373867 41.899324573751)
3,0b5603954d84b7341c7cad4f570ea039e85919f3750ccb...,NaN,05/23/2025 09:15:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PEDALCYCLIST,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,Y,NaN,NaN,$500 OR LESS,05/23/2025 09:16:00 AM,UNABLE TO DETERMINE,UNABLE TO DETERMINE,2901,W,DIVISION ST,1211.0,NaN,NaN,NaN,NaN,NaN,NaN,2,NONINCAPACITATING INJURY,1.0,0.0,0.0,1.0,0.0,2.0,0.0,9,6,5,41.902793,-87.699412,POINT (-87.699412181285 41.902792968177)
4,00bce77960c2faa2a8782a8cac1d6e5715802d6c072a08...,NaN,01/14/2025 08:00:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,SNOW,DAYLIGHT,REAR END,FOUR WAY,NaN,STRAIGHT AND LEVEL,WET,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,Y,NaN,NaN,"OVER $1,500",01/14/2025 08:45:00 AM,FOLLOWING TOO CLOSELY,FOLLOWING TOO CLOSELY,4001,W,111TH ST,2211.0,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,8,3,1,41.691207,-87.720555,POINT (-87.720554863466 41.691206664451)


## Data Cleaning

### lowercasing the column names

In [6]:
traffic_crashes.columns = traffic_crashes.columns.str.lower()

### dropping unnecessary columns

In [7]:
traffic_crashes.drop(columns=(['crash_record_id', 'crash_date_est_i']), inplace=True)

### lowercasing rows to improve readibilty

In [8]:
# .lower() can be applied to strings only, strings are assigned object dtype
# so I first select columns of object dtype
# then, I create a lambda function to lowercase the rows in those columns
# i used x.str.lower and not x.lower because .apply() iterates column by column and not element-wise when a dataframe is passed in 

string_columns = traffic_crashes.select_dtypes(include=['object'])
traffic_crashes[string_columns.columns] = string_columns.apply(lambda x: x.str.lower())

### modifying categorical values for some columns

In [79]:
traffic_crashes['road_defect'] = traffic_crashes['road_defect'].replace(
    {'unknown' : 'other'}
)

In [83]:
traffic_crashes['prim_contributory_cause'] = traffic_crashes['prim_contributory_cause'].replace(
    {
    'vision obscured (signs, tree limbs, buildings, etc.)' : 'distraction - from outside vehicle',
    'animal' : 'distraction - from outside vehicle',
    'distraction - other electronic device (navigation device, dvd player, etc.)' : 'distraction - from inside vehicle',
    'disregarding other traffic signs' : 'disregarding traffic signs',
    'disregarding yield sign' : 'disregarding traffic signs',
    'turning right on red' : 'disregarding traffic signals'
    }
)

### identifying and dropping duplicates

In [93]:
dup_rows = traffic_crashes[traffic_crashes.duplicated()]

In [94]:
traffic_crashes.drop(dup_rows.index, inplace=True)

## EDA

### what is the number of of incidents where police was notified in over 3 days after the incident?

In [9]:
# first, converting 'crash date' and 'date police notified' to datetime64[ns]
# automatic format assigned by pandas caused error, so I specified it myself

traffic_crashes['crash_date'] = pd.to_datetime(traffic_crashes['crash_date'], format='%m/%d/%Y %I:%M:%S %p')
traffic_crashes['date_police_notified'] = pd.to_datetime(traffic_crashes['date_police_notified'], format='%m/%d/%Y %I:%M:%S %p')

In [10]:
# then, I subtract the columns

traffic_crashes['how quickly police was notified'] = traffic_crashes['date_police_notified'] - traffic_crashes['crash_date']

In [11]:
traffic_crashes[['crash_date', 'date_police_notified', 'how quickly police was notified']].head()

,crash_date,date_police_notified,how quickly police was notified
0,2025-01-14 12:25:00,2025-01-14 12:38:00,0 days 00:13:00
1,2025-05-23 09:30:00,2025-05-27 10:40:00,4 days 01:10:00
2,2025-04-05 20:00:00,2025-04-05 21:23:00,0 days 01:23:00
3,2025-05-23 09:15:00,2025-05-23 09:16:00,0 days 00:01:00
4,2025-01-14 08:00:00,2025-01-14 08:45:00,0 days 00:45:00


In [12]:
len(traffic_crashes[traffic_crashes['how quickly police was notified'] > '3 days'])

42641

### for each weather condition, what is the highest number of total injuries recorded?

In [13]:
traffic_crashes.groupby('weather_condition')['injuries_total'].max().sort_values(ascending=False)

weather_condition
clear                       21.0
rain                        21.0
snow                        15.0
other                       12.0
cloudy/overcast             11.0
blowing snow                10.0
unknown                     10.0
sleet/hail                   6.0
freezing rain/drizzle        6.0
fog/smoke/haze               5.0
severe cross wind gate       4.0
blowing sand, soil, dirt     2.0
Name: injuries_total, dtype: float64

### what is the number of incidents recorded for each weekday?

In [14]:
crash_counts = traffic_crashes['crash_day_of_week'].value_counts()

In [15]:
day_mappings = {
    1: 'Monday',
    2: 'Tuesday',
    3: 'Wednesday',
    4: 'Thursday',
    5: 'Friday',
    6: 'Saturday',
    7: 'Sunday'
}

In [16]:
crash_counts_with_names = crash_counts.rename(index=day_mappings)

In [17]:
crash_counts_with_names.sort_values(ascending=False)

crash_day_of_week
Saturday     162381
Sunday       147520
Friday       144714
Wednesday    142688
Thursday     141933
Tuesday      137368
Monday       125129
Name: count, dtype: int64

### for each primary contributory cause, what is the most common damage value?

In [18]:
# I also excluded 'unable to determine' group as it isn't informative enough

traffic_crashes[~traffic_crashes['prim_contributory_cause'].isin(['unable to determine'])].groupby('prim_contributory_cause')['damage'].agg(lambda x: x.mode())

prim_contributory_cause
animal                                                                              over $1,500
bicycle advancing legally on red light                                              over $1,500
cell phone use other than texting                                                   over $1,500
disregarding other traffic signs                                                    over $1,500
disregarding road markings                                                          over $1,500
disregarding stop sign                                                              over $1,500
disregarding traffic signals                                                        over $1,500
disregarding yield sign                                                             over $1,500
distraction - from inside vehicle                                                   over $1,500
distraction - from outside vehicle                                                  over $1,500
distraction - ot

### what is the most dangerous hour?

In [23]:
traffic_crashes.groupby('crash_hour').size().sort_values(ascending=False)

crash_hour
15    77841
16    76813
17    74382
14    66873
18    61181
13    60448
12    58402
8     53411
11    50744
9     45823
10    45472
19    45181
7     42802
20    36714
21    32764
22    30038
23    26141
0     21836
6     21623
1     18660
2     15997
5     13765
3     13188
4     11634
dtype: int64

### output the 2021 incidents that occured on sunday at dawn, causing over 1 total injuries

In [37]:
traffic_crashes[
    (traffic_crashes['lighting_condition'] == 'dawn') & 
    (traffic_crashes['crash_day_of_week'] == 7) &  
    (traffic_crashes['injuries_total'] > 1) &
    (traffic_crashes['crash_date'].dt.year == 2021)
]

,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,lane_cnt,alignment,roadway_surface_cond,road_defect,report_type,crash_type,intersection_related_i,not_right_of_way_i,hit_and_run_i,damage,date_police_notified,prim_contributory_cause,sec_contributory_cause,street_no,street_direction,street_name,beat_of_occurrence,photos_taken_i,statements_taken_i,dooring_i,work_zone_i,work_zone_type,workers_present_i,num_units,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,injuries_unknown,crash_hour,crash_day_of_week,crash_month,latitude,longitude,location,how quickly police was notified
62221,2021-09-18 20:34:00,30,traffic signal,functioning properly,clear,dawn,angle,four way,NaN,straight and level,dry,no defects,on scene,injury and / or tow due to crash,y,NaN,NaN,"over $1,500",2021-09-18 20:58:00,unable to determine,unable to determine,800,e,63rd st,313.0,NaN,NaN,NaN,NaN,NaN,NaN,2,nonincapacitating injury,2.0,0.0,0.0,2.0,0.0,2.0,0.0,20,7,9,41.780521,-87.605869,point (-87.605869499248 41.780521024744),0 days 00:24:00
162860,2021-04-24 04:00:00,30,traffic signal,functioning properly,clear,dawn,rear end,divided - w/median (not raised),NaN,straight and level,dry,no defects,not on scene (desk report),injury and / or tow due to crash,y,NaN,NaN,"over $1,500",2021-04-24 22:30:00,physical condition of driver,physical condition of driver,5500,s,hamlin ave,822.0,NaN,NaN,NaN,NaN,NaN,NaN,2,nonincapacitating injury,3.0,0.0,0.0,3.0,0.0,1.0,0.0,4,7,4,41.793229,-87.718406,point (-87.718406157995 41.793229125263),0 days 18:30:00
335060,2021-06-05 05:19:00,35,traffic signal,functioning properly,clear,dawn,angle,four way,NaN,straight and level,dry,unknown,on scene,injury and / or tow due to crash,y,NaN,NaN,"over $1,500",2021-06-05 05:19:00,disregarding traffic signals,failing to yield right-of-way,7600,s,state st,623.0,NaN,NaN,NaN,NaN,NaN,NaN,2,nonincapacitating injury,2.0,0.0,0.0,2.0,0.0,2.0,0.0,5,7,6,41.756400,-87.624870,point (-87.624869685896 41.756400406673),0 days 00:00:00
458883,2021-11-06 00:45:00,30,traffic signal,unknown,clear,dawn,rear end,divided - w/median (not raised),NaN,straight and level,dry,unknown,on scene,injury and / or tow due to crash,NaN,NaN,y,"over $1,500",2021-11-06 00:45:00,failing to reduce speed to avoid crash,following too closely,11500,s,avenue o,433.0,NaN,NaN,NaN,NaN,NaN,NaN,3,"reported, not evident",3.0,0.0,0.0,0.0,3.0,1.0,0.0,0,7,11,41.686360,-87.540261,point (-87.540261369531 41.686360380338),0 days 00:00:00
635153,2021-02-06 12:15:00,30,no controls,no controls,cloudy/overcast,dawn,turning,not divided,NaN,straight and level,snow or slush,no defects,on scene,injury and / or tow due to crash,y,NaN,NaN,"over $1,500",2021-02-06 12:15:00,unable to determine,unable to determine,3459,w,belmont ave,1412.0,NaN,NaN,NaN,NaN,NaN,NaN,2,incapacitating injury,3.0,0.0,1.0,1.0,1.0,0.0,0.0,12,7,2,41.939137,-87.714877,point (-87.714876767242 41.93913679392),0 days 00:00:00
760174,2021-08-14 23:53:00,30,no controls,no controls,clear,dawn,fixed object,not divided,NaN,straight and level,dry,unknown,on scene,injury and / or tow due to crash,NaN,NaN,NaN,"over $1,500",2021-08-14 23:53:00,failing to reduce speed to avoid crash,failing to reduce speed to avoid crash,2100,e,103rd st,431.0,NaN,NaN,NaN,NaN,NaN,NaN,1,"reported, not evident",2.0,0.0,0.0,0.0,2.0,0.0,0.0,23,7,8,41.708073,-87.573863,point (-87.573862566855 41.708073049807),0 days 00:00:00
811289,2021-10-02 20:52:00,35,traffic signal,functioning properly,rain,dawn,angle,four way,NaN,straight and level,wet,no defects,on scene,injury and / or tow due to crash,y,NaN,NaN,"over $1,500",2021-10-02 20:58:00,failing to yield right-of-way,failing to yield right-of-way,7900,s,stony island ave,411.0,NaN,NaN,NaN,NaN,NaN,NaN,2,nonincapacitating injury,2.0,0.0,0.0,1.0,1.0,2.0,0.0,20,7,10,41.751461,-87.585972,point 

### what is the percentage of missing values per column?

In [72]:
traffic_crashes.isnull().sum()*100/len(traffic_crashes)

crash_date                          0.000000
posted_speed_limit                  0.000000
traffic_control_device              0.000000
device_condition                    0.000000
weather_condition                   0.000000
lighting_condition                  0.000000
first_crash_type                    0.000000
trafficway_type                     0.000000
lane_cnt                           80.131233
alignment                           0.000000
roadway_surface_cond                0.000000
road_defect                         0.000000
report_type                         3.299881
crash_type                          0.000000
intersection_related_i             77.018926
not_right_of_way_i                 95.505888
hit_and_run_i                      68.620880
damage                              0.000000
date_police_notified                0.000000
prim_contributory_cause             0.000000
sec_contributory_cause              0.000000
street_no                           0.000000
street_dir